In [1]:
import pandas as pd
import numpy as np
import requests
import re
import os
from time import sleep
from tqdm import tqdm
from functools import reduce

import datetime

In [11]:
BASE_GOOGLE_FINANCE_URL = "https://api.polygon.io/v2/aggs/ticker/{TICKER}/range/{INTERVAL}/" +\
                          "{RANGE}/{FROM_DATE}/{TO_DATE}?limit=50000&apiKey=MNxlbyNXPpKmAki1jcScqmXxzPvdXWYT"

# TOKENS = [
#     "AAPL", "GOOG", "MSFT", "TSLA", "NVDA", "AMZN",
#     "FB", "BABA", "CRM", "INTC", "AMD", "PYPL", "ATVI",
#     "EA", "TTD", "MTCH", "ZG", "YELP"
# ]
TOKENS = ["X:BTCUSD"]

TIME_INTERVAL = 15
TIME_UNITS = "minute"
FROM_DATE = pd.Timestamp("2018-02-18").date()
TO_DATE = pd.Timestamp("2022-04-11").date()

DATASET_NAME = "BTC"

FULL_RESULT_PATH = f"prices/{DATASET_NAME}_{TIME_INTERVAL}{TIME_UNITS}.csv"
TRAIN_RESULT_PATH = f"prices/{DATASET_NAME}_{TIME_INTERVAL}{TIME_UNITS}_train.csv"
TEST_RESULT_PATH = f"prices/{DATASET_NAME}_{TIME_INTERVAL}{TIME_UNITS}_test.csv"

In [12]:
def get_url(token: str, interval_len: int, interval: str, from_date: datetime.date, to_date: datetime.date):
    """
    Build url for Google Finance API
    """
    url = re.sub("{TICKER}", token, BASE_GOOGLE_FINANCE_URL)
    url = re.sub("{INTERVAL}", str(interval_len), url)
    url = re.sub("{RANGE}", interval, url)
    url = re.sub("{FROM_DATE}", str(from_date), url)
    url = re.sub("{TO_DATE}", str(to_date), url)
    return url

In [13]:
def get_data(token: str, interval_len: int, interval: str, from_date: datetime.date, to_date: datetime.date):
    TIMEOUT = 20
    current_date = from_date
    previous_date = current_date
    dfs = []
    while current_date <= to_date:
        request_url = get_url(token, interval_len, interval, current_date, to_date)
        data = requests.get(url=request_url)
        if data.json()["status"] == "ERROR":
            print("Waiting for", TIMEOUT, "seconds")
            for _ in tqdm(range(TIMEOUT)):
                sleep(1)
        else:
            try:
                current_df = pd.DataFrame(data.json()["results"])
            except:
                pass
            current_date = (pd.Timestamp(current_df["t"].max(), unit="ms") + pd.DateOffset(days=1)).date()
            print("Current date:", current_date)
            print("Get DataFrame of shape", current_df.shape)
            dfs.append(current_df)
    result_df = pd.concat(dfs)
    result_df.drop_duplicates(inplace=True)
    return result_df

In [14]:
dfs = []
for token in TOKENS:
    data = get_data(
        token=token,
        interval_len=TIME_INTERVAL,
        interval=TIME_UNITS,
        from_date=FROM_DATE, 
        to_date=TO_DATE
    )

    data["t"] = data["t"].apply(lambda row: pd.Timestamp(row, unit="ms"))
    data["ticker"] = token
    dfs.append(data)

Waiting for 20 seconds


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:20<00:00,  1.00s/it]


Current date: 2020-05-19
Get DataFrame of shape (3333, 8)
Current date: 2020-06-23
Get DataFrame of shape (3333, 8)
Current date: 2020-07-28
Get DataFrame of shape (3333, 8)
Current date: 2020-09-01
Get DataFrame of shape (3333, 8)
Current date: 2020-10-06
Get DataFrame of shape (3333, 8)
Waiting for 20 seconds


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:20<00:00,  1.01s/it]


Waiting for 20 seconds


...

Waiting for 20 seconds


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:20<00:00,  1.01s/it]


Waiting for 20 seconds


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:20<00:00,  1.01s/it]


Current date: 2022-04-12
Get DataFrame of shape (2592, 8)


In [8]:
RENAME_COLUMNS = {
    "v": "volume",
    "o": "open",
    "c": "close",
    "h": "high",
    "l": "low",
    "t": "date"
}
KEEP_COLUMNS = ["volume", "open", "close", "high", "low", "date", "ticker"]

result_df = pd.concat(dfs).rename(RENAME_COLUMNS, axis=1)[KEEP_COLUMNS].reset_index(drop=True)
print(result_df.shape)
result_df

(1036000, 7)


,volume,open,close,high,low,date,ticker
0,19.242033,7387.23,7383.66,7393.17,7368.68,2020-04-08 23:43:00,X:BTCUSD
1,18.150259,7380.30,7379.70,7387.25,7367.68,2020-04-08 23:44:00,X:BTCUSD
2,18.564893,7380.00,7374.40,7386.49,7365.78,2020-04-08 23:45:00,X:BTCUSD
3,74.808246,7364.26,7373.59,7380.21,7362.68,2020-04-08 23:46:00,X:BTCUSD
4,43.765348,7375.00,7366.50,7377.00,7355.39,2020-04-08 23:47:00,X:BTCUSD
...,...,...,...,...,...,...,...
1035995,4.936814,46621.95,46597.42,46621.96,46584.52,2022-04-04 23:55:00,X:BTCUSD
1035996,2.237617,46600.00,46613.03,46615.95,46591.14,2022-04-04 23:56:00,X:BTCUSD
1035997,8.357742,46613.03,46617.90,46626.59,46609.00,2022-04-04 23:57:00,X:BTCUSD
1035998,14.586789,46617.89,46622.49,46638.80,46612.45,2022-04-04 23:58:00,X:BTCUSD


In [10]:
dfs

[]

In [9]:
TEST_RATIO = 0.2
test_size = int(result_df.shape[0] * TEST_RATIO)
data_train = result_df.iloc[:-test_size]
data_test = result_df.iloc[-test_size:]

result_df.to_csv(FULL_RESULT_PATH)
data_train.to_csv(TRAIN_RESULT_PATH)
data_test.to_csv(TEST_RESULT_PATH)

In [15]:
test_size

207200